## Orcid API

Have three different types of APIs available for use. For the purposes of this tutorial, the Public API will be used. Both the Member API and the Public API for Orcid uses two-legged authorization. 

Two-legged authorization: A call is sent to Orcid API containing credential information and the API returns a token. The access token returned has a 20 year life span and can be used to perform searches and accessing other Orcid records. 

In [ ]:
#run the following on you CLI
curl -i -L -H "Accept: application/json" -d "client_id=APP-NPXKK6HFN6TJ4YYI" -d "client_secret=060c36f2-cce2-4f74-bde0-a17d8bb30a97" -d "scope=/read-public" -d "grant_type=client_credentials" "https://sandbox.orcid.org/oauth/token" >> toxen.txt

In a Unix based system, the curl request above can be easily performed. If you are using windows, [installing](https://superuser.com/questions/134685/run-curl-commands-from-windows-console)curl maybe useful for future projects. 

This should result in a file containing a JSON object holding access tokens. The result I received is the following:

In [ ]:
HTTP/1.1 200 OK
Date: Sun, 14 Jan 2018 20:19:33 GMT
Content-Type: application/json;charset=UTF-8
Transfer-Encoding: chunked
Connection: keep-alive
Set-Cookie: __cfduid=df6e73b8a367baa24602ca455f996426e1515961173; expires=Mon, 14-Jan-19 20:19:33 GMT; path=/; domain=.orcid.org; HttpOnly
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Pragma: no-cache
Expires: 0
X-XSS-Protection: 1; mode=block
X-Frame-Options: DENY
X-Content-Type-Options: nosniff
Server: cloudflare
CF-RAY: 3dd34a76de2c27e0-YYZ

{"access_token":"79bd2af3-8079-454e-9807-031545a9acac","token_type":"bearer","refresh_token":"03eb52fd-2fe3-4278-bd44-77a65b652cc9","expires_in":631138518,"scope":"/read-public","orcid":null}

### Searching by institution

When using the Public API, two source URLs ([Sandbox](https://pub.sandbox.orcid.org) vs. [Production](https://pub.orcid.org)) are available. This part of the tutorial will use the Production URL and perform a GET request which is expected to return an data in an XML format. 

The API is under constant development. As a result, the base url for API is followed by the version extension. 

In [25]:
head = {"access_token":"79bd2af3-8079-454e-9807-031545a9acac"}
ins = 'University of Waterloo'
url = 'https://pub.orcid.org/v2.1/search/?q=%s' % ins.replace(" ","+")
r = requests.get(url, headers=head)
r.text #shows the returned result in XML format

https://pub.orcid.org/v2.1/search/?q=University+of+Waterloo/orcid-bio/


'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<search:search num-found="1858742" xmlns:search="http://www.orcid.org/ns/search" xmlns:common="http://www.orcid.org/ns/common">\n    <search:result>\n        <common:orcid-identifier>\n            <common:uri>https://orcid.org/0000-0002-6405-6172</common:uri>\n            <common:path>0000-0002-6405-6172</common:path>\n            <common:host>orcid.org</common:host>\n        </common:orcid-identifier>\n    </search:result>\n    <search:result>\n        <common:orcid-identifier>\n            <common:uri>https://orcid.org/0000-0003-1102-1520</common:uri>\n            <common:path>0000-0003-1102-1520</common:path>\n            <common:host>orcid.org</common:host>\n        </common:orcid-identifier>\n    </search:result>\n    <search:result>\n        <common:orcid-identifier>\n            <common:uri>https://orcid.org/0000-0002-5465-446X</common:uri>\n            <common:path>0000-0002-5465-446X</common:path>\n            <common:h

The search method shown above can become quiet powerful as the url is concatenated with author names and other parameters such as Boolean values. The ability to also take a list of institutions, iterate through them, and generating a file containing Orcid data can also be useful. 

**You Try**
Modify the code in the above URL to search by McLevy AND University of Waterloo. 

In [33]:
#Below is a sample code - does your data match with this?
head = {"access_token":"79bd2af3-8079-454e-9807-031545a9acac"}
auth_name = 'McLevey'
ins = 'University of Waterloo'
url = 'https://pub.orcid.org/v2.1/search/?q=%s+AND+%s' % (auth_name, ins.replace(" ","+"))
r = requests.get(url, headers=head) 
r.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<search:search num-found="1" xmlns:search="http://www.orcid.org/ns/search" xmlns:common="http://www.orcid.org/ns/common">\n    <search:result>\n        <common:orcid-identifier>\n            <common:uri>https://orcid.org/0000-0002-8512-1308</common:uri>\n            <common:path>0000-0002-8512-1308</common:path>\n            <common:host>orcid.org</common:host>\n        </common:orcid-identifier>\n    </search:result>\n</search:search>\n'

The above record should show only one record for the given criteria. To match this record, you can always visit the [advanced search](https://orcid.org/orcid-search/search) feature on Orcid and validate. 

### Search by known ID

If you know the ORCID of a researcher and would like to fetch the necessary information a similar approach to above can be taken. This trick can be more powerful if there is a list of known IDs and you would like iterate through the list for each individual and collect data as a JSON object. 

## Working with a Wrapper

An API wrapper is a library that increases a user's accessibility to an exisiting third-party API. The ORCID-Python wrapper library is an example of that. A Python wrapper is expected to return data in formats that are friendly for further manipulation. Please note that according to [official](https://pypi.python.org/pypi/orcid/) doc, this wrapper library has been perfected for the sandbox. Take caution about using this in future projects (specailly in a production environment). 

In [12]:
import orcid
cn = orcid.SearchAPI("0000-0002-0068-716X")
